In [38]:
import pint
import math

ureg = pint.UnitRegistry(auto_reduce_dimensions=True)

In [207]:
#@ureg.wraps('[length] / [time]', ('[length]', '1 / [time]'))
#@ureg.wraps(ureg.millimeter / ureg.minute, (ureg.mm, 1 / ureg.second))
@ureg.check('[length]', 'turn / [time]')
def cutting_speed(cutter_diameter, rpm):
    D_c = cutter_diameter
    n = rpm
    v_c = D_c * math.pi * n
    
    assert(v_c.check('[length] / [time]'))
    
    return v_c # unit / min

cutter_diameter = 12.7 * ureg.mm
rpm = 1000 * ureg.revolutions_per_minute

v_c = cutting_speed(cutter_diameter, rpm)

print(cutter_diameter, rpm, v_c, sep='\n')

12.7 millimeter
1000 revolutions_per_minute
39898.22670059037 millimeter * revolutions_per_minute


In [202]:
cutter_diameter = 12.7 * ureg.mm
rpm = 1000 * ureg.revolutions_per_minute
print(rpm.dimensionality)
rpm = 1000 * ureg.turn / ureg.min
print(rpm.dimensionality)

v_c = cutting_speed(cutter_diameter, rpm)

print(v_c)
print(v_c.to_compact())
print(v_c.to_base_units())
print(v_c.to_reduced_units())
print(v_c.to(ureg.m / ureg.min))
print(v_c.to(ureg.inch / ureg.min))
print(v_c.dimensionality)
print(v_c.check('[length] / [time]'))
# print(type(v_c.dimensionality))
# print(ureg.check('[length] / [time]')(v_c))
#print(v_c.dimensionality == ureg.length / ureg.time)

print()

cutter_diameter = 0.5 * ureg.inch
rpm = 1000 / ureg.min

v_c = cutting_speed(cutter_diameter, rpm)
print(v_c)
print(v_c.to_compact())
print(v_c.to_base_units())
print(v_c.to_reduced_units())
print(v_c.to(ureg.m / ureg.min))
print(v_c.to(ureg.inch / ureg.min))

1 / [time]
1 / [time]
250687.95178766965 millimeter / minute
True
250687.95178766965 millimeter / minute
250.68795178766965 meter / minute
4.178132529794494 meter / second
250687.95178766965 millimeter / minute
250.68795178766965 meter / minute
9869.604401089357 inch / minute
[length] / [time]
True

1570.7963267948965 inch / minute
True
1570.7963267948965 inch / minute
1.5707963267948966 kiloinch / minute
0.6649704450098395 meter / second
1570.7963267948965 inch / minute
39.89822670059037 meter / minute
1570.7963267948965 inch / minute


In [141]:
a = 1 * ureg.revolutions_per_minute
b = 1 * ureg.turn / ureg.minute
print(a)
print(a.dimensionality)
print(b)
print(b.dimensionality)

print(a.check('1 / [time]'))
print(a.check('revolutions / [time]'))


1 revolutions_per_minute
1 / [time]
1.0 turn / minute
1 / [time]
True
True


In [230]:
@ureg.check('[length]', '[length] / [time]')
def spindle_speed(cutter_diameter, cutting_speed):
    D_c = cutter_diameter
    v_c = cutting_speed
    n = v_c / (math.pi * D_c)
    print(n)

    # n.ito('revolutions_per_minute') by itself will divide by 2Pi, but change units to rev_per_min
    # *= ureg.turn only adds the unit
    # *= ureg.turn and n.ito('rev...') will not change value and change units to rev_per_min
    n *= ureg.turn
    n.ito('revolutions_per_minute')
    assert(n.check('turn / [time]'))
    
    return n # rpm

cutter_diameter = 12.7 * ureg.mm
cutting_speed = 40 * ureg.m / ureg.min

n = spindle_speed(cutter_diameter, cutting_speed)
print(n)
print(f'{n:.2f}')

1002.5508226261124 / minute
1002.5508226261124 revolutions_per_minute
1002.55 revolutions_per_minute


In [239]:
@ureg.check('[length] / turn', 'turn / [time]')
def penetration_rate(speed_per_revolution, spindle_speed):
    f_n = speed_per_revolution
    n = spindle_speed
    v_f = f_n * n
    
    assert(v_f.check('[length] / [time]'))
    
    return v_f # unit / min
    
# speed_per_revolution = 1 * ureg.mm / ureg.revolution
# spindle_speed = 1000 * ureg.revolutions_per_minute

# using turn instead of rev_per_min results in simplier final units, without need for to_base_units()
speed_per_revolution = 1 * ureg.mm / ureg.turn
spindle_speed = 1000 * (ureg.turn / ureg.minute)

v_f = penetration_rate(speed_per_revolution, spindle_speed)

print(v_f)
# print(v_f.to_compact())
# print(v_f.to_base_units())
# print(v_f.to_reduced_units())
# print(v_f.to(ureg.mm / ureg.min))

1000.0 millimeter / minute


In [241]:
# @ureg.wraps('[length] / turn',            (ureg.mile / ureg.min, ureg.revolutions_per_minute))
@ureg.check('[length] / [time]', 'revolutions_per_minute')
def feed_per_revolution(penetration_rate, spindle_speed):
    v_f = penetration_rate
    n = spindle_speed
    f_n = v_f / n
    
    # convert from radians to turns
    f_n *= 2 * math.pi / ureg.turn
    # Check return type; Pint can only currently force the return type with @wraps()
    assert(f_n.check('[length] / turn'))
    
    return f_n # mm / rev

penetration_rate = 1000 * ureg.mm / ureg.min
spindle_speed = 1000 * ureg.revolutions_per_minute

f_n = feed_per_revolution(penetration_rate, spindle_speed)

print(penetration_rate, spindle_speed)
print(f_n)

# print((1 * ureg.revolutions_per_minute) * (1 * ureg.minute))
# print((1 * ureg.rpm) * (1 * ureg.minute))
# print((1 * ureg.turn / (1 * ureg.minute)) * (1 * ureg.minute))
# print((1 * ureg.revolution / (1 * ureg.minute)) * (1 * ureg.minute))
# print(((1 * ureg.revolution / (1 * ureg.minute)) * (1 * ureg.minute)).to(ureg.turn))

1000.0 millimeter / minute 1000 revolutions_per_minute
1.0 millimeter / turn


In [242]:
@ureg.check('[length]', '[length] / turn', '[length] / [time]')
def metal_removal_rate(cutter_diameter, feed_per_revolution, cutting_speed):
    D_c = cutter_diameter
    f_n = feed_per_revolution
    v_c = cutting_speed
    # / 4 
    Q = D_c * f_n * v_c / 4
    
    Q *= ureg.turn
    print(Q)
    print(Q.check('[length] ** 2 / [time] / turn'))
    print(Q.check('[length] ** 3 / [time] / turn'))
    print(Q.check('[volume] / [time] / turn'))
    print(Q.check('[length] ** 3 / [time]'))
    print(Q.check('[volume] / [time]'))
    
    return Q # cm^3 / min

cutter_diameter = 12.7 * ureg.mm
feed_per_revolution = 1 * ureg.mm / ureg.turn
cutting_speed = 1000 * ureg.mm / ureg.min

Q = metal_removal_rate(cutter_diameter, feed_per_revolution, cutting_speed)

print(Q)

3175.0 millimeter ** 3 / minute
False
True
True
True
True
3175.0 millimeter ** 3 / minute


In [194]:
print((1 * (ureg.turn / ureg.min)) * (1 * (ureg.mm / ureg.turn)))

1 millimeter / minute


In [285]:
@ureg.check('[length]', '[length] / turn', 'turn / [time]')
def metal_removal_rate(cutter_diameter, feed_per_rev, spindle_rpm):
    f = feed_per_rev # [mm / rev]
    N = spindle_rpm # rev / min
    D_c = cutter_diameter

    f_r = f * N # feed_rate [mm / min]
    # The /4 comes from r^2 = (d/2)^2
    Q = (math.pi * D_c**2 / 4) * f_r
    
#     Q *= ureg.turn
    print(Q)
    print(Q.check('[length] ** 2 / [time] / turn'))
    print(Q.check('[length] ** 3 / [time] / turn'))
    print(Q.check('[volume] / [time] / turn'))
    print(Q.check('[length] ** 3 / [time]'))
    print(Q.check('[volume] / [time]'))
    
    return Q # cm^3 / min

cutter_diameter = 12.7 * ureg.mm
feed_per_revolution = 1 * ureg.mm / ureg.turn
# spindle_speed = 1000 * ureg.revolutions_per_minute
spindle_speed = 1000 * (ureg.turn / ureg.minute)

Q = metal_removal_rate(cutter_diameter, feed_per_revolution, spindle_speed)

print(Q)
print(Q.to('cm^3 / min'))

126676.86977437443 millimeter ** 3 / minute
False
True
True
True
True
126676.86977437443 millimeter ** 3 / minute
126.67686977437442 centimeter ** 3 / minute


In [281]:
@ureg.check('[length]', '[length] / turn', '[length] / [time]', '[force] / [area]')
def net_power(cutter_diameter, feed_per_revolution, cutting_speed, specific_cutting_force):
    D_c = cutter_diameter
    f_n = feed_per_revolution
    v_c = cutting_speed
    k_c = specific_cutting_force
    P_c = (f_n * v_c * D_c * k_c) / (240.)
    
    assert(P_c.check('watt'))
    P_c.ito('watt')
    
    return P_c # kW

cutter_diameter = 12.7 * ureg.mm
feed_per_revolution = .2 * (ureg.mm / ureg.turn)
cutting_speed = 75 * (ureg.m / ureg.min)
cutting_speed = 75 * 1000 * (ureg.mm / ureg.min)
specific_cutting_force = 350 * (ureg.newton / ureg.mm ** 2)
# specific_cutting_force.to('kilowatt / cm**3 / min')

print(specific_cutting_force.to_base_units())
# P = net_power(cutter_diameter, feed_per_revolution, cutting_speed, specific_cutting_force)

# print(P)

350000000.0 kilogram / meter / second ** 2


In [302]:
@ureg.check('[length]', '[length] / turn', 'turn / [time]', '[power] / ([volume] / [time])')
def net_power(cutter_diameter, feed_per_revolution, spindle_rpm, specific_cutting_energy):
    Q = metal_removal_rate(cutter_diameter, feed_per_revolution, spindle_rpm)
    u_s = specific_cutting_energy

    P = Q * u_s
    
#     print(Q)
#     print(u_s)
#     print(P)
    assert(P.check('watt'))
#     P.ito('watt')
    
    return P # kW

cutter_diameter = 12.7 * ureg.mm
feed_per_revolution = .2 * (ureg.mm / ureg.turn)
spindle_rpm = 1000 * (ureg.turn / ureg.min)
# specific_cutting_energy = .065 * (ureg.kilowatt / ureg.cm**3 / ureg.min)
# print(specific_cutting_energy)
specific_cutting_energy = .065 * (ureg.kilowatt / (ureg.cm**3 / ureg.min))
print(specific_cutting_energy)
P = net_power(cutter_diameter, feed_per_revolution, spindle_rpm, specific_cutting_energy)

print(P)

0.065 kilowatt * minute / centimeter ** 3
25335.373954874885 millimeter ** 3 / minute
False
True
True
True
True
1.6467993070668676 kilowatt


In [11]:
def torque(net_power, spindle_speed):
    P_c = net_power
    n = spindle_speed
    M_c = (P_c * 30 * 10**3) / (math.pi * n)
    return M_c # N m


def specific_cutting_force():
    k_c = 0.
    return k_c # N / mm^2


def feed_force():
    F_f = 0.
    return F_f # N


def machining_time(I_m, penetration_rate):
    I_m = I_m
    v_f = penetration_rate
    T_c = I_m / v_f
    return T_c # min


ureg = pint.UnitRegistry(auto_reduce_dimensions=True)

cutter_diameter = 12.7 * ureg.mm
cutting_speed = 76 * ureg.m / ureg.min
rpm = spindle_speed(cutter_diameter, cutting_speed)
print(rpm)
print(rpm.to_compact())
    
print(cutter_diameter, rpm, cutting_speed(cutter_diameter, rpm))

spindle_speed = 1000 / ureg.min
speed_per_revolution = 1 * ureg.mm

print(penetration_rate(speed_per_revolution, spindle_speed))


3032265.2292448683 millimeter ** 2 / minute
3.032265229244868 meter ** 2 / minute


TypeError: 'Quantity' object is not callable